# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Kevin Martínez Gallego
#### Segundo Integrante: Andrés Mauricio Álvarez Ortiz

In [18]:
from __future__ import division

%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
import numpy.matlib as matlib

#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Para este ejercicio usaremos la base de datos sobre calidad del aire, que ha sido usada en laboratorios previos, pero en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [19]:
#cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter

#Esta es la base de datos AirQuality del UCI Machine Learning Repository. En la siguiente URL se encuentra toda
#la descripción de la base de datos y la contextualización del problema.
#https://archive.ics.uci.edu/ml/datasets/Air+Quality#

X = db[:,0:11]
Y = db[:,11:13]

In [20]:
print(X.shape)
print(Y.shape)

(9357, 11)
(9357, 2)


In [21]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape

## Ejercicio 1

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [22]:
from numpy import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing

def MLPR(layers, neurons):
    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    ErrorY1 = np.zeros(Folds)
    ErrorY2 = np.zeros(Folds)
    ErrorT = np.zeros(Folds)
    ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
    j = 0
    for train, test in ss.split(X):
        Xtrain = X[train,:]
        Ytrain = Y[train,:]
        Xtest = X[test,:]
        Ytest = Y[test,:]

        #Normalizamos los datos
        media = np.mean(Xtrain,axis=0)
        desvia = np.std(Xtrain,axis=0)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        epochs = 500
        if(layers == 1): numberOfNeurons = (neurons,)
        elif (layers == 2): numberOfNeurons = (neurons,neurons)
        
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp = MLPRegressor(hidden_layer_sizes=numberOfNeurons, activation='tanh', max_iter=epochs).fit(Xtrain, Ytrain)

        #Use para el modelo para hacer predicciones sobre el conjunto Xtest
        Yest = mlp.predict(Xtest)

        #Mida el error MAPE para cada una de las dos salidas
        ErrorY1[j] = MAPE(Yest[:,0], Ytest[:,0])
        ErrorY2[j] = MAPE(Yest[:,1], Ytest[:,1])
        ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
        j += 1

    print('MAPE salida 1 = ' + str(np.mean(ErrorY1)) + '+-' + str(np.std(ErrorY1)))
    print('MAPE salida 2 = ' + str(np.mean(ErrorY2)) + '+-' + str(np.std(ErrorY2)))
    print('MAPE total = '    + str(np.mean(ErrorT)) + '+-' + str(np.std(ErrorT)))
    return (np.mean(ErrorY1), np.std(ErrorY1), np.mean(ErrorY2), np.std(ErrorY2))

## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [25]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["MAPE salida 1"] = ""
df_types["IC MAPE salida 1"] = ""
df_types["MAPE salida 2"] = ""
df_types["IC MAPE salida 2"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'])
#df_types.sort_index(inplace=True)

index = 0
maxLayers = 2
neurons = [20,24,28,32,36]
for i in range(maxLayers):
    layers = i+1
    for j in range(np.size(neurons)):
        MAPEY1, ICY1, MAPEY2, ICY2 = MLPR(layers, neurons[j])
        df_types["MAPE salida 1"][index] = MAPEY1
        df_types["IC MAPE salida 1"][index] = ICY1
        df_types["MAPE salida 2"][index] = MAPEY2
        df_types["IC MAPE salida 2"][index]= ICY2
        index += 1
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

MAPE salida 1 = 0.08622996152522724+-0.0019551111123171025
MAPE salida 2 = 2.3985811386210187+-0.15854819303283926
MAPE total = 1.2424055500731233+-0.07903364336697818


C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

MAPE salida 1 = 0.08184301090778363+-0.0018665481639035385
MAPE salida 2 = 2.0614908461444568+-0.10960088265898164
MAPE total = 1.0716669285261202+-0.05432244126955209


C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Kevin\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

...

MAPE salida 1 = 0.06363067028563801+-0.0010719398643725224
MAPE salida 2 = 0.8684634682710998+-0.08841852785970206
MAPE total = 0.4660470692783689+-0.04408451270774313


C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value i

MAPE salida 1 = 0.0625755976134793+-0.0021234707146164083
MAPE salida 2 = 0.8577153437027267+-0.013428989534007592
MAPE total = 0.460145470658103+-0.006928463994991388


C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value i

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [27]:
qgrid_widget.get_changed_df()

,N. de capas ocultas,Neuronas por capa,MAPE salida 1,IC MAPE salida 1,MAPE salida 2,IC MAPE salida 2
0,1,20,0.08623,0.00195511,2.39858,0.158548
1,1,24,0.081843,0.00186655,2.06149,0.109601
2,1,28,0.0772476,0.00135356,1.73297,0.0694282
3,1,32,0.0768586,0.00173177,1.51695,0.0508611
4,1,36,0.0750917,0.000896848,1.50108,0.0389907
5,2,20,0.0714244,0.00312684,1.2208,0.0997434
6,2,24,0.0664214,0.00231797,0.984584,0.107353
7,2,28,0.0652746,0.000813113,0.82862,0.0993152
8,2,32,0.0636307,0.00107194,0.868463,0.0884185
9,2,36,0.0625756,0.00212347,0.857715,0.013429


<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

Se utiliza una función identity, la cual no modifica la salida del modelo.

## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación de dígitos escritos a mano. Usaremos únicamente 4 de las 10 clases disponibles. Los datos fueron preprocesados para reducir el número de características. La técnica usada será analizada más adelante en el curso.

In [28]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

In [29]:
print(X.shape)
print(Y.shape)
print(np.unique(Y, return_counts = True))

(720, 39)
(720,)
(array([0, 1, 2, 3]), array([178, 182, 177, 183], dtype=int64))


Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

def MLPC(layers, neurons):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        epochs = 500
        
        if(layers == 1): numberOfNeurons = (neurons,)
        elif (layers == 2): numberOfNeurons = (neurons,neurons)
        
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp = MLPClassifier(hidden_layer_sizes=numberOfNeurons, activation='tanh', max_iter=epochs).fit(Xtrain, Ytrain)

        #Validación con las muestras de entrenamiento
        Ytrain_pred = mlp.predict(Xtrain)

        #Validación con las muestras de test    
        Yest = mlp.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1

    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    return(np.mean(EficienciaVal), np.std(EficienciaVal))

In [13]:
MLPC(1, 24)

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9623501551552112+-0.033708835273301894


(0.9623501551552112, 0.033708835273301894)

## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [31]:
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'])
#df_types.sort_index(inplace=True)

index = 0
maxLayers = 2
neurons = [20,24,28,32,36]
for i in range(maxLayers):
    layers = i+1
    for j in range(np.size(neurons)):
        eff, std = MLPC(layers, neurons[j])
        df_types["Eficiencia en validacion"][index] = round(eff, 4)
        df_types["Intervalo de confianza"][index] = round(std, 4)
        index += 1

qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9567941737434934+-0.026823128593228423


C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9623501551552112+-0.033708835273301894


C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9637852603042869+-0.03034887331847802


...

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9610155003686535+-0.025302076371385318


C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9680065897876841+-0.021431415557815768


C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [32]:
qgrid_widget.get_changed_df()

,N. de capas ocultas,Neuronas por capa,Eficiencia en validacion,Intervalo de confianza
0,1,20,0.9568,0.0268
1,1,24,0.9624,0.0337
2,1,28,0.9638,0.0303
3,1,32,0.9652,0.0246
4,1,36,0.9651,0.0312
5,2,20,0.968,0.0265
6,2,24,0.9569,0.0294
7,2,28,0.9652,0.0284
8,2,32,0.961,0.0253
9,2,36,0.968,0.0214


<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 


Softmax


4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Porqué debe tener ese número?

El modelo tiene 4 neuronas en la capa de salida. Esto se debe a que estamos abordando un problema con 4 clases, tal que se genera una salida para cada una.